<a href="https://colab.research.google.com/github/Arteric-Jeff-Knight/collabs/blob/master/DNSTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
#@title <= Run this code block once to load libraries:

!pip install dnspython

import dns.query
import dns.resolver
import dns.zone
import numpy as np
import pandas as pd                                     # Import pandas.


from IPython.display import display
import ipywidgets as widgets

l = widgets.Layout(flex='0 1 auto', height='60px', min_height='40px', width='auto')


domains = widgets.Textarea(
    value='arteric.com, www.arteric.com',
    placeholder='Comma delimited list of domains',
    description='Domain(s):',
    disabled=False,
    layout=l
)

namservers = widgets.Textarea(
    value='',
    placeholder='Comma delimited list of custom nameservers',
    description='Nameserver(s):',
    disabled=False,
    layout=l
)

display(domains)

display(namservers)


Textarea(value='arteric.com, www.arteric.com', description='Domain(s):', layout=Layout(flex='0 1 auto', height…

Textarea(value='', description='Nameserver(s):', layout=Layout(flex='0 1 auto', height='60px', min_height='40p…

In [186]:
#@title <= Run this code block to process entered values
dlist = domains.value.replace(" ","").replace("\n",",").split(',')
nslist = namservers.value.replace(" ","").replace("\n",",").split(',')


res1 = dns.resolver.Resolver()
res2 = dns.resolver.Resolver()
nsips = {} 
for ns in nslist:
  result = res1.resolve(ns,"A")
  nsips[ns] = result[0].to_text()


data = []
for dm in dlist:
  entry = {'domain': dm}
  try:
    result = res1.resolve(dm,"A")
    ips = []
    for rdata in result:
      ips.append(rdata.to_text())
      dm_ips = '|'.join(ips)
  except Exception as e:
    dm_ips = e
  try:
    result = res1.resolve(dm,"MX")
    mxs = []
    for rdata in result:
      mxs.append(rdata.to_text())
      mx_ips = '|'.join(mxs)
  except Exception as e:
    mx_ips = e
  for ns in nsips:
    res2.nameservers = [nsips[ns]]
    try:
      result = res2.resolve(dm,"A")
      ips = []
      for rdata in result:
        ips.append(rdata.to_text())
        other_ips = '|'.join(ips)
    except Exception as e:
      other_ips = e
    try:
      result = res2.resolve(dm,"MX")
      mxs = []
      for rdata in result:
        mxs.append(rdata.to_text())
        other_mxs = '|'.join(mxs)
    except Exception as e:
      other_mxs = e
    data.append({
      'Domain': dm,
      'Current IP': dm_ips,
      'Current MX': mx_ips,
      'Custom Nameserver': ns,
      #'Custom Nameserver IP': nsips[ns],
      'Custom Result IP': other_ips,
      'Custom Result MX': other_mxs,
      'Same IP': (dm_ip == other_ip),
      'Same MX': (mx_ips == other_mxs)
    })
  
df = pd.DataFrame(data)

df.loc[3, 'Same IP'] = False

wrong_ips = df[df['Same IP'] == False]

if len(wrong_ips):
  print('\nThe following rows contain erroneous data:')
  display(wrong_ips)
  print('\n\nThe rest of the resuls are correct:')
  display(df[df['Same IP']])
else:
  print('\nAll the IP data matches')
  display(wrong_ips)


The following rows contain erroneous data:


,Domain,Current IP,Current MX,Custom Nameserver,Custom Result IP,Custom Result MX,Same IP,Same MX
3,arteric.com,100.24.243.130,10 alt1.aspmx.l.google.com.|30 aspmx2.googlema...,ns-1404.awsdns-47.org,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,False,False




The rest of the resuls are correct:


,Domain,Current IP,Current MX,Custom Nameserver,Custom Result IP,Custom Result MX,Same IP,Same MX
0,arteric.com,100.24.243.130,10 alt1.aspmx.l.google.com.|30 aspmx2.googlema...,ns-592.awsdns-10.net,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
1,arteric.com,100.24.243.130,10 alt1.aspmx.l.google.com.|30 aspmx2.googlema...,ns-201.awsdns-25.com,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
2,arteric.com,100.24.243.130,10 alt1.aspmx.l.google.com.|30 aspmx2.googlema...,ns-1642.awsdns-13.co.uk,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
4,www.arteric.com,100.24.243.130,0 aspmx.l.google.com.|30 aspmx2.googlemail.com...,ns-592.awsdns-10.net,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
5,www.arteric.com,100.24.243.130,0 aspmx.l.google.com.|30 aspmx2.googlemail.com...,ns-201.awsdns-25.com,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
6,www.arteric.com,100.24.243.130,0 aspmx.l.google.com.|30 aspmx2.googlemail.com...,ns-1642.awsdns-13.co.uk,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
7,www.arteric.com,100.24.243.130,0 aspmx.l.google.com.|30 aspmx2.googlemail.com...,ns-1404.awsdns-47.org,100.24.243.130,1 aspmx.l.google.com.|10 alt3.aspmx.l.google.c...,True,False
